<a href="https://colab.research.google.com/github/Alokik-29/Project_2/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('hey')

hey


In [2]:
"""
TEXT-TO-SQL RAG PROJECT
Comparing Llama 3.1 vs SQLCoder on Chinook Database

Author: Your Name
Date: October 2025
"""

# ============================================================================
# SECTION 1: SETUP & INSTALLATION
# ============================================================================

print("="*80)
print("SECTION 1: Installing Dependencies")
print("="*80)

!pip install -q transformers accelerate bitsandbytes sentence-transformers chromadb
!pip install -q torch datasets huggingface_hub

print("✅ Installation complete!")

# ============================================================================
# SECTION 2: DOWNLOAD CHINOOK DATABASE
# ============================================================================

print("\n" + "="*80)
print("SECTION 2: Downloading Chinook Database")
print("="*80)

import urllib.request
import sqlite3
import os

# Download database
db_url = "https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
db_path = "chinook.db"

if not os.path.exists(db_path):
    print("Downloading Chinook database...")
    urllib.request.urlretrieve(db_url, db_path)
    print("✅ Database downloaded!")
else:
    print("✅ Database already exists!")

# Verify database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

print(f"\n📊 Found {len(tables)} tables in Chinook database:")
for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
    count = cursor.fetchone()[0]
    print(f"  - {table[0]}: {count} rows")

conn.close()

# ============================================================================
# SECTION 3: EXTRACT DATABASE SCHEMA
# ============================================================================

print("\n" + "="*80)
print("SECTION 3: Extracting Database Schema")
print("="*80)

import sqlite3
import json

def extract_schema(db_path):
    """Extract complete database schema with columns and relationships"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    schema_info = {}

    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [table[0] for table in cursor.fetchall()]

    for table in tables:
        # Get column information
        cursor.execute(f"PRAGMA table_info({table})")
        columns = cursor.fetchall()

        # Get foreign keys
        cursor.execute(f"PRAGMA foreign_key_list({table})")
        foreign_keys = cursor.fetchall()

        # Get sample data (1 row)
        cursor.execute(f"SELECT * FROM {table} LIMIT 1")
        sample = cursor.fetchone()

        schema_info[table] = {
            'columns': [
                {
                    'name': col[1],
                    'type': col[2],
                    'primary_key': bool(col[5]),
                    'sample': sample[col[0]] if sample else None
                }
                for col in columns
            ],
            'foreign_keys': [
                {
                    'column': fk[3],
                    'references_table': fk[2],
                    'references_column': fk[4]
                }
                for fk in foreign_keys
            ]
        }

    conn.close()
    return schema_info

# Extract schema
schema = extract_schema(db_path)

# Display schema summary
print(f"\n📋 Schema extracted for {len(schema)} tables:")
for table_name, info in schema.items():
    print(f"\n  {table_name}:")
    print(f"    Columns: {', '.join([col['name'] for col in info['columns']])}")
    if info['foreign_keys']:
        print(f"    Foreign Keys: {len(info['foreign_keys'])}")

# Save schema
with open('schema.json', 'w') as f:
    json.dump(schema, f, indent=2)

print("\n✅ Schema saved to schema.json")

# ============================================================================
# SECTION 4: CREATE VECTOR STORE FOR SCHEMA (RAG Component)
# ============================================================================

print("\n" + "="*80)
print("SECTION 4: Creating Vector Store for Schema Retrieval")
print("="*80)

from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

# Initialize embedding model
print("Loading embedding model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize ChromaDB
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(
    name="database_schema",
    metadata={"description": "Chinook database schema"}
)

# Prepare schema documents for embedding
documents = []
metadatas = []
ids = []

for table_name, info in schema.items():
    # Create schema description
    col_descriptions = []
    for col in info['columns']:
        desc = f"{col['name']} ({col['type']})"
        if col['primary_key']:
            desc += " [PRIMARY KEY]"
        col_descriptions.append(desc)

    # Create foreign key descriptions
    fk_descriptions = []
    for fk in info['foreign_keys']:
        fk_descriptions.append(
            f"{fk['column']} references {fk['references_table']}.{fk['references_column']}"
        )

    # Combine into document
    doc = f"Table: {table_name}\n"
    doc += f"Columns: {', '.join(col_descriptions)}\n"
    if fk_descriptions:
        doc += f"Foreign Keys: {', '.join(fk_descriptions)}\n"

    documents.append(doc)
    metadatas.append({"table": table_name})
    ids.append(f"table_{table_name}")

# Add to vector store
print(f"Adding {len(documents)} table schemas to vector store...")
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print("✅ Vector store created!")

# Test retrieval
test_query = "Find tables related to customer purchases and orders"
results = collection.query(
    query_texts=[test_query],
    n_results=3
)

print(f"\n🔍 Test Query: '{test_query}'")
print("Top 3 relevant tables:")
for i, doc in enumerate(results['documents'][0]):
    print(f"\n{i+1}. {results['metadatas'][0][i]['table']}")
    print(f"   {doc[:200]}...")

# ============================================================================
# SECTION 5: FEW-SHOT EXAMPLES
# ============================================================================

print("\n" + "="*80)
print("SECTION 5: Creating Few-Shot Examples")
print("="*80)

# Few-shot examples to help LLM understand the task
few_shot_examples = [
    {
        "question": "List all artists",
        "sql": "SELECT * FROM Artist LIMIT 10;"
    },
    {
        "question": "Show me the top 5 customers by total spending",
        "sql": """SELECT c.CustomerId, c.FirstName, c.LastName, SUM(i.Total) as TotalSpent
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY TotalSpent DESC
LIMIT 5;"""
    },
    {
        "question": "What are the most popular genres by number of tracks?",
        "sql": """SELECT g.Name as Genre, COUNT(t.TrackId) as TrackCount
FROM Genre g
JOIN Track t ON g.GenreId = t.GenreId
GROUP BY g.GenreId
ORDER BY TrackCount DESC;"""
    },
    {
        "question": "Find all albums by AC/DC",
        "sql": """SELECT al.Title as AlbumTitle
FROM Album al
JOIN Artist ar ON al.ArtistId = ar.ArtistId
WHERE ar.Name = 'AC/DC';"""
    }
]

print(f"✅ Created {len(few_shot_examples)} few-shot examples")

# ============================================================================
# SECTION 6: LOAD MODELS FROM HUGGING FACE
# ============================================================================

print("\n" + "="*80)
print("SECTION 6: Loading Models from Hugging Face")
print("="*80)

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Quantization config to fit models in Colab free tier
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# ============================================================================
# LOAD MODEL 1: SQLCoder
# ============================================================================

print("\n📥 Loading SQLCoder...")
sqlcoder_name = "defog/sqlcoder-7b-2"

try:
    sqlcoder_tokenizer = AutoTokenizer.from_pretrained(sqlcoder_name)
    sqlcoder_model = AutoModelForCausalLM.from_pretrained(
        sqlcoder_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    print("✅ SQLCoder loaded successfully!")
except Exception as e:
    print(f"❌ Error loading SQLCoder: {e}")
    print("Trying alternative model...")
    sqlcoder_name = "defog/sqlcoder2"
    sqlcoder_tokenizer = AutoTokenizer.from_pretrained(sqlcoder_name)
    sqlcoder_model = AutoModelForCausalLM.from_pretrained(
        sqlcoder_name,
        quantization_config=bnb_config,
        device_map="auto"
    )

# ============================================================================
# LOAD MODEL 2: Llama 3.1 (or alternative if access restricted)
# ============================================================================

print("\n📥 Loading Llama model...")

# Try Llama 3.1 first, fallback to other models if access denied
llama_models = [
    "meta-llama/Meta-Llama-3.1-8B-Instruct",  # Primary choice
    "meta-llama/Llama-2-7b-chat-hf",           # Fallback 1
    "codellama/CodeLlama-7b-Instruct-hf",      # Fallback 2
    "mistralai/Mistral-7B-Instruct-v0.2"       # Fallback 3
]

llama_model = None
llama_tokenizer = None
llama_name = None

for model_name in llama_models:
    try:
        print(f"Trying {model_name}...")
        llama_tokenizer = AutoTokenizer.from_pretrained(model_name)
        llama_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True
        )
        llama_name = model_name
        print(f"✅ {model_name} loaded successfully!")
        break
    except Exception as e:
        print(f"⚠️ Could not load {model_name}: {str(e)[:100]}")
        continue

if llama_model is None:
    print("❌ Could not load any Llama model. Please check HuggingFace access.")
else:
    print(f"\n✅ Both models loaded successfully!")
    print(f"   Model 1: {sqlcoder_name}")
    print(f"   Model 2: {llama_name}")

# ============================================================================
# SECTION 7: CREATE SQL GENERATION FUNCTIONS
# ============================================================================

print("\n" + "="*80)
print("SECTION 7: Creating SQL Generation Functions")
print("="*80)

def retrieve_relevant_schema(query, n_results=3):
    """Retrieve relevant table schemas based on query"""
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    return results['documents'][0]

def create_prompt(question, schema_context, few_shot=True):
    """Create prompt for SQL generation"""
    prompt = "You are a SQL expert. Generate a SQLite query to answer the question.\n\n"

    # Add few-shot examples
    if few_shot:
        prompt += "Here are some examples:\n\n"
        for ex in few_shot_examples[:2]:  # Use 2 examples
            prompt += f"Question: {ex['question']}\n"
            prompt += f"SQL: {ex['sql']}\n\n"

    # Add schema context
    prompt += "Database Schema:\n"
    for schema_doc in schema_context:
        prompt += f"{schema_doc}\n"

    # Add the actual question
    prompt += f"\nQuestion: {question}\n"
    prompt += "SQL: "

    return prompt

def generate_sql_sqlcoder(question, max_length=500):
    """Generate SQL using SQLCoder"""
    # Retrieve relevant schema
    schema_context = retrieve_relevant_schema(question)

    # Create prompt
    prompt = create_prompt(question, schema_context, few_shot=True)

    # Generate
    inputs = sqlcoder_tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = sqlcoder_model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.1,
            do_sample=False,
            pad_token_id=sqlcoder_tokenizer.eos_token_id
        )

    # Decode
    full_response = sqlcoder_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract SQL (everything after "SQL: ")
    if "SQL: " in full_response:
        sql = full_response.split("SQL: ")[-1].strip()
    else:
        sql = full_response.strip()

    # Clean up SQL
    sql = sql.split('\n\n')[0]  # Take only first query
    sql = sql.strip()

    return sql

def generate_sql_llama(question, max_length=500):
    """Generate SQL using Llama"""
    # Retrieve relevant schema
    schema_context = retrieve_relevant_schema(question)

    # Create prompt
    prompt = create_prompt(question, schema_context, few_shot=True)

    # Generate
    inputs = llama_tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = llama_model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.1,
            do_sample=False,
            pad_token_id=llama_tokenizer.eos_token_id
        )

    # Decode
    full_response = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract SQL
    if "SQL: " in full_response:
        sql = full_response.split("SQL: ")[-1].strip()
    else:
        sql = full_response.strip()

    # Clean up SQL
    sql = sql.split('\n\n')[0]
    sql = sql.strip()

    return sql

def execute_sql(sql, db_path="chinook.db"):
    """Execute SQL and return results"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql)
        results = cursor.fetchall()
        column_names = [description[0] for description in cursor.description]
        conn.close()
        return {"success": True, "results": results, "columns": column_names}
    except Exception as e:
        return {"success": False, "error": str(e)}

print("✅ SQL generation functions created!")

# ============================================================================
# SECTION 8: TEST QUERIES
# ============================================================================

print("\n" + "="*80)
print("SECTION 8: Testing Both Models")
print("="*80)

# Test queries
test_queries = [
    "List the top 5 artists with most albums",
    "What is the total revenue from invoices in 2010?",
    "Show me all customers from Canada",
    "Find the longest track in the database",
    "Which employee has the most customers?"
]

results_comparison = []

for i, question in enumerate(test_queries, 1):
    print(f"\n{'='*80}")
    print(f"TEST QUERY {i}: {question}")
    print('='*80)

    # Generate SQL with SQLCoder
    print("\n🤖 SQLCoder generating...")
    sql_sqlcoder = generate_sql_sqlcoder(question)
    print(f"SQLCoder SQL:\n{sql_sqlcoder}")

    # Execute SQLCoder query
    result_sqlcoder = execute_sql(sql_sqlcoder)
    if result_sqlcoder['success']:
        print(f"✅ SQLCoder query executed successfully!")
        print(f"   Returned {len(result_sqlcoder['results'])} rows")
    else:
        print(f"❌ SQLCoder query failed: {result_sqlcoder['error']}")

    # Generate SQL with Llama
    print(f"\n🤖 {llama_name.split('/')[-1]} generating...")
    sql_llama = generate_sql_llama(question)
    print(f"Llama SQL:\n{sql_llama}")

    # Execute Llama query
    result_llama = execute_sql(sql_llama)
    if result_llama['success']:
        print(f"✅ Llama query executed successfully!")
        print(f"   Returned {len(result_llama['results'])} rows")
    else:
        print(f"❌ Llama query failed: {result_llama['error']}")

    # Store results
    results_comparison.append({
        'question': question,
        'sqlcoder_sql': sql_sqlcoder,
        'sqlcoder_success': result_sqlcoder['success'],
        'llama_sql': sql_llama,
        'llama_success': result_llama['success']
    })

    print()

# ============================================================================
# SECTION 9: EVALUATION & COMPARISON
# ============================================================================

print("\n" + "="*80)
print("SECTION 9: Model Comparison Results")
print("="*80)

# Calculate metrics
sqlcoder_success_rate = sum(1 for r in results_comparison if r['sqlcoder_success']) / len(results_comparison) * 100
llama_success_rate = sum(1 for r in results_comparison if r['llama_success']) / len(results_comparison) * 100

print(f"\n📊 OVERALL METRICS:")
print(f"   Total Test Queries: {len(results_comparison)}")
print(f"   SQLCoder Success Rate: {sqlcoder_success_rate:.1f}%")
print(f"   Llama Success Rate: {llama_success_rate:.1f}%")

print(f"\n{'='*80}")
print("DETAILED COMPARISON:")
print('='*80)

for i, result in enumerate(results_comparison, 1):
    print(f"\n{i}. {result['question']}")
    print(f"   SQLCoder: {'✅ Success' if result['sqlcoder_success'] else '❌ Failed'}")
    print(f"   Llama:    {'✅ Success' if result['llama_success'] else '❌ Failed'}")

# Determine winner
if sqlcoder_success_rate > llama_success_rate:
    winner = "SQLCoder"
    diff = sqlcoder_success_rate - llama_success_rate
elif llama_success_rate > sqlcoder_success_rate:
    winner = "Llama"
    diff = llama_success_rate - sqlcoder_success_rate
else:
    winner = "Tie"
    diff = 0

print(f"\n🏆 WINNER: {winner}")
if diff > 0:
    print(f"   Margin: {diff:.1f}% higher success rate")

print("\n" + "="*80)
print("PROJECT COMPLETE! 🎉")
print("="*80)
print("\nYou can now:")
print("1. Try your own queries using generate_sql_sqlcoder() and generate_sql_llama()")
print("2. Compare the generated SQL queries")
print("3. Execute them using execute_sql()")
print("4. Extend with more test cases")
print("5. Add a Streamlit UI for better visualization")

# ============================================================================
# SECTION 10: INTERACTIVE TESTING (Optional)
# ============================================================================

print("\n" + "="*80)
print("SECTION 10: Interactive Testing")
print("="*80)

def test_custom_query(question):
    """Test a custom query with both models"""
    print(f"\n📝 Question: {question}")
    print("="*80)

    # SQLCoder
    print("\n🤖 SQLCoder:")
    sql1 = generate_sql_sqlcoder(question)
    print(f"SQL: {sql1}")
    result1 = execute_sql(sql1)
    if result1['success']:
        print(f"✅ Success! Returned {len(result1['results'])} rows")
        if result1['results']:
            print(f"Columns: {result1['columns']}")
            print(f"First 3 rows: {result1['results'][:3]}")
    else:
        print(f"❌ Error: {result1['error']}")

    # Llama
    print(f"\n🤖 Llama:")
    sql2 = generate_sql_llama(question)
    print(f"SQL: {sql2}")
    result2 = execute_sql(sql2)
    if result2['success']:
        print(f"✅ Success! Returned {len(result2['results'])} rows")
        if result2['results']:
            print(f"Columns: {result2['columns']}")
            print(f"First 3 rows: {result2['results'][:3]}")
    else:
        print(f"❌ Error: {result2['error']}")

print("\n💡 Try testing your own queries!")
print("Example:")
print("  test_custom_query('Show me all rock albums')")
print("  test_custom_query('What is the average invoice total?')")

# Example usage
test_custom_query("Show me the top 3 most expensive tracks")

SECTION 1: Installing Dependencies
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✅ codellama/CodeLlama-7b-Instruct-hf loaded successfully!

✅ Both models loaded successfully!
   Model 1: defog/sqlcoder-7b-2
   Model 2: codellama/CodeLlama-7b-Instruct-hf

SECTION 7: Creating SQL Generation Functions
✅ SQL generation functions created!

SECTION 8: Testing Both Models

TEST QUERY 1: List the top 5 artists with most albums

🤖 SQLCoder generating...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


SQLCoder SQL:
SELECT a.ArtistId, a.Name, COUNT(al.AlbumId) AS AlbumCount FROM Artist a JOIN Album al ON a.ArtistId = al.ArtistId GROUP BY a.ArtistId, a.Name ORDER BY AlbumCount DESC LIMIT 5;
✅ SQLCoder query executed successfully!
   Returned 5 rows

🤖 CodeLlama-7b-Instruct-hf generating...
Llama SQL:

❌ Llama query failed: 'NoneType' object is not iterable


TEST QUERY 2: What is the total revenue from invoices in 2010?

🤖 SQLCoder generating...
SQLCoder SQL:
SELECT SUM(i.Total) AS TotalRevenue FROM Invoice i WHERE EXTRACT(YEAR FROM i.InvoiceDate) = 2010;
❌ SQLCoder query failed: near "FROM": syntax error

🤖 CodeLlama-7b-Instruct-hf generating...
Llama SQL:
Question: What is the total revenue from invoices in 2
❌ Llama query failed: near "Question": syntax error


TEST QUERY 3: Show me all customers from Canada

🤖 SQLCoder generating...
SQLCoder SQL:
SELECT c.CustomerId, c.FirstName, c.LastName, c.Country FROM Customer c WHERE c.Country = 'Canada';
✅ SQLCoder query executed successful